In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

df_chamado = spark.read.table("v_credit.silver.tb_chamado")
df_custo = spark.read.table("v_credit.silver.tb_custo_chamado")

TABELA_DESTINO = "v_credit.gold.ft_custo_operacional"

### Join e Criacao do Fato de Custos
Faz INNER JOIN entre tb_chamado e tb_custo_chamado e cria tabela fato especializada:
- **pk_custo**: Surrogate key (MD5 hash)
- **FKs**: cd_chamado, cd_motivo, cd_canal
- **Metrica**: val_custo_total (DECIMAL 12,10)

Faz MERGE em ft_custo_operacional (Gold).

**Importante**: Fato agregado usado para analise de Pareto de custos por motivo e canal (identifica desperdicio de 29%).

### Configuracao e Leitura
Le tabelas Silver:
- tb_chamado (para FKs: cd_motivo, cd_canal)
- tb_custo_chamado (para metrica vl_custo)

In [0]:
df_join = df_chamado.alias("c").join(df_custo.alias("cust"), "cd_chamado")

df_gold = df_join.select(

    F.md5(F.concat(F.col("c.cd_chamado").cast("string"), F.lit("CUSTO"))).alias("pk_custo"),
    
    F.col("c.cd_chamado").cast("bigint").alias("cd_chamado"),
    F.col("c.cd_motivo"),
    F.col("c.cd_canal"),
    
    F.col("cust.vl_custo").cast("decimal(12,10)").alias("val_custo_total"),
    
    F.to_date("c.dt_ingestion").alias("dt_referencia")
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.pk_custo = s.pk_custo"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Fato Custo Operacional carregada (Linhagem preservada de chamados)!")